Exploration of Tradeable Search API endpoint
docs: https://www.alphavantage.co/documentation/#listing-status

observations:
- with no parameters it lists active securities on latest trading day
- only USA securities

In [1]:
# setup

import os
import io
import requests
import pandas as pd
from rich import print
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv('.env.dev.local'))
api_key = os.environ['alphavantage_api_key']
endpoint = 'https://www.alphavantage.co/query?function=LISTING_STATUS'

# No parameters

In [2]:
request_url = f'{endpoint}&apikey={api_key}'
response = requests.get(request_url)
csv_data = response.content.decode('utf-8')
df_default = pd.read_csv(io.StringIO(csv_data))
df_default

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active
...,...,...,...,...,...,...,...
11664,ZYME,Zymeworks BC Inc,NASDAQ,Stock,2017-04-28,NaN,Active
11665,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,NaN,Active
11666,ZYXI,Zynex Inc,NASDAQ,Stock,2012-07-23,NaN,Active
11667,ZZK,NaN,NYSE ARCA,Stock,2020-07-22,NaN,Active


In [3]:
# make sure it's only active securities
# status should all be Active and delistingDate NaN
unique_status = df_default['status'].unique()
unique_del_date = df_default['delistingDate'].unique()
print('status: ', unique_status)
print('del_date :', unique_del_date)

status:  ['Active']

del_date : [nan]

In [4]:
# count of active stocks on NSYE and NASDAQ
filtered_df_default = df_default[ (df_default['assetType'] == 'Stock') & (df_default['exchange'].isin(['NYSE', 'NASDAQ'])) ]
filtered_df_default.count()

symbol           7167
name             7141
exchange         7168
assetType        7168
ipoDate          7168
delistingDate       0
status           7168
dtype: int64

# Parameter state=delisted

In [5]:
request_url = f'{endpoint}&state=delisted&apikey={api_key}'
response = requests.get(request_url)
csv_data = response.content.decode('utf-8')
df_delisted = pd.read_csv(io.StringIO(csv_data))
df_delisted

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,AA-W,Alcoa Corporation When Issued,NYSE,Stock,2016-10-18,2016-11-08,Delisted
1,AAAP,Advanced Accelerator Applications SA,NASDAQ,Stock,2015-11-11,2018-02-20,Delisted
2,AABA,Altaba Inc,NASDAQ,Stock,1996-04-12,2019-11-06,Delisted
3,AAC,American Addiction Centers,NYSE,Stock,2014-10-02,2021-04-19,Delisted
4,AAC-U,Ares Acquisition Corporation - Units (1 Ord Sh...,NYSE,Stock,2021-02-02,2023-11-06,Delisted
...,...,...,...,...,...,...,...
8363,ZWRK,Z-Work Acquisition Corp - Class A,NASDAQ,Stock,2021-04-05,2022-12-08,Delisted
8364,ZWRKU,Z-Work Acquisition Corp - Units (1 Ord Class A...,NASDAQ,Stock,2021-01-29,2022-12-08,Delisted
8365,ZWRKW,Z-Work Acquisition Corp - Warrants (29/01/2026),NASDAQ,Stock,2021-03-24,2022-12-08,Delisted
8366,ZX,China Zenix Auto International Ltd,NYSE,Stock,2011-05-16,2018-06-14,Delisted


In [6]:
unique_status = df_delisted['status'].unique()
unique_status

array(['Delisted'], dtype=object)

In [7]:
filtered_df_delisted = df_delisted[(df_delisted['assetType'] == 'Stock') & (df_delisted['exchange'].isin(['NYSE', 'NASDAQ']))]
filtered_df_delisted.count()

symbol           6596
name             6464
exchange         6596
assetType        6596
ipoDate          6596
delistingDate    6596
status           6596
dtype: int64

# Misc explorations

In [17]:
# count of merged data frames
df_combined = pd.concat([df_default, df_delisted], ignore_index=True)
df_combined.count()

symbol           20036
name             19868
exchange         20037
assetType        20037
ipoDate          20037
delistingDate     8368
status           20037
dtype: int64

In [21]:
# see if there are duplicate symbols. There shouldn't be any
df_combined[df_combined.duplicated(subset='symbol', keep=False)].sort_values(by='symbol')

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
54,ABST,Absolute Software Corporation,NASDAQ,Stock,2020-10-28,NaN,Active
11718,ABST,Absolute Software Corp,NASDAQ,Stock,2007-07-13,2020-10-28,Delisted
68,ACACU,Acri Capital Acquisition Corp - Units (1 Ord C...,NASDAQ,Stock,2022-06-10,NaN,Active
11724,ACACU,PLAYSTUDIOS Inc - Units (1 Ord Share Class A &...,NASDAQ,Stock,2020-10-23,2021-06-21,Delisted
71,ACB,Aurora Cannabis Inc,NASDAQ,Stock,2014-07-11,NaN,Active
...,...,...,...,...,...,...,...
11460,XONE,BONDBLOXX BLOOMBERG ONE YEAR TARGET DURATION U...,NYSE ARCA,ETF,2022-09-15,NaN,Active
19930,XRTX,Xyratex Ltd,NASDAQ,Stock,2006-12-28,2014-03-31,Delisted
11485,XRTX,XORTX Therapeutics Inc,NASDAQ,Stock,2018-11-28,NaN,Active
11586,ZBIO,Zenas Biopharma Inc,NASDAQ,Stock,2024-09-13,NaN,Active
